In [1]:
# create pyoxigraph store

from pyoxigraph import *

store = Store()
store.clear()

In [2]:
ttl_file = open("../turtle-files/representation_history.ttl", "rb")
store.load(ttl_file, "text/turtle", base_iri="http://example.org/")
# list(store)

In [3]:
# show all representations

showAllReps = """
    PREFIX : <http://example.org/>
    SELECT ?s WHERE {
        ?s a :Representation
    }
    ORDER BY ASC(?s)
"""
list(store.query(showAllReps))

[<QuerySolution s=<NamedNode value=http://example.org/picture_02>>,
 <QuerySolution s=<NamedNode value=http://example.org/picture_a>>,
 <QuerySolution s=<NamedNode value=http://example.org/picture_b>>,
 <QuerySolution s=<NamedNode value=http://example.org/scan_01>>]

In [4]:
# show all active picture representations

showAllActiveReps = """
    PREFIX : <http://example.org/>
    SELECT ?rep WHERE {
        << ?rep a :Representation >>
            :representationType :picture .

        << << ?element :representedBy ?rep >> :hasStatus "active" >> :date ?latestTime .
        {
            SELECT (MAX(?creationDate) as ?latestTime) WHERE {
                << ?element :representedBy ?rep >> 
                    :hasStatus ?status .

                << << ?element :representedBy ?rep >> :hasStatus ?status >> :date ?creationDate .
            }
        }
    }
    ORDER BY ASC(?rep)
"""
list(store.query(showAllActiveReps))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_02>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_b>>]

In [5]:
# show all picture representations and their latest status

allPicsAndStatus = """
    PREFIX : <http://example.org/>
    SELECT ?rep ?status WHERE {
        << ?rep a :Representation >>
            :representationType :picture .

        << << ?element :representedBy ?rep >> :hasStatus ?status >>  :date ?latestTime .
        {
            SELECT ?el ?rep (MAX(?creationDate) as ?latestTime) WHERE {
                << ?element :representedBy ?rep >> 
                    :hasStatus ?status .

                << << ?element :representedBy ?rep >> :hasStatus ?status >> :date ?creationDate .
            }
            GROUP BY ?el ?rep
        }
    }
    ORDER BY ?rep ?status
"""
list(store.query(allPicsAndStatus))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_02> status=<Literal value=active datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_a> status=<Literal value=inactive datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_b> status=<Literal value=active datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>]

In [6]:
# show all status changes of representation regarding their represented element

query = """
    PREFIX : <http://example.org/>
    SELECT ?element ?status ?date WHERE {
        ?element :representedBy :picture_01 .
        << ?element :representedBy :picture_01 >> :hasStatus ?status .
        << << ?element :representedBy :picture_01 >> :hasStatus ?status >> :date ?date .
    }
    ORDER BY DESC(?date)
"""
list(store.query(query))

[<QuerySolution element=<NamedNode value=http://example.org/column_01> status=<Literal value=inactive datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>> date=<Literal value=2021-08-09T15:05:47.277000000015433728Z datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#date>>>,
 <QuerySolution element=<NamedNode value=http://example.org/column_01> status=<Literal value=active datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>> date=<Literal value=2021-04-09T15:05:47.277000000015433728Z datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#date>>>]

In [7]:
# show all representations of element

query = """
    PREFIX : <http://example.org/>
    SELECT ?rep WHERE {
        ?rep a :Representation . 
        :column_01 :representedBy ?rep . 
    }
    ORDER BY ASC(?rep)
"""
list(store.query(query))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_02>>,
 <QuerySolution rep=<NamedNode value=http://example.org/scan_01>>]

In [8]:
# show all active picture representations of element

allActivePicRepsOfElement = """
    PREFIX : <http://example.org/>
    SELECT ?rep WHERE {
        BIND (:column_01 as ?element)

        << ?rep a :Representation >>
            :representationType :picture .

        << << ?element :representedBy ?rep >> :hasStatus "active" >>  :date ?latestTime .
        {
            SELECT ?el ?rep (MAX(?creationDate) as ?latestTime) WHERE {
                << ?element :representedBy ?rep >> 
                    :hasStatus ?status .

                << << ?element :representedBy ?rep >> :hasStatus ?status >> :date ?creationDate .
            }
            GROUP BY ?el ?rep
        }
    }
    ORDER BY ASC(?rep)
"""
list(store.query(allActivePicRepsOfElement))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_02>>]

In [9]:
# get one previous version of representation

query = """
    PREFIX : <http://example.org/>
    SELECT ?prevRep WHERE {
        << :picture_02 a :Representation >>
            :hasPreviousRepresentation ?prevRep .
    }
    ORDER BY ASC(?prevRep)
"""
list(store.query(query))

[<QuerySolution prevRep=<Triple subject=<NamedNode value=http://example.org/picture_01> predicate=<NamedNode value=http://www.w3.org/1999/02/22-rdf-syntax-ns#type> object=<NamedNode value=http://example.org/Representation>>>]

In [10]:
# get name of all previous representations

query = """
    PREFIX : <http://example.org/>
    SELECT (SUBJECT(?prevRep) as ?result) WHERE {
        << :picture_02 a :Representation >>
            :hasPreviousRepresentation* ?prevRep .
        # BIND( SUBJECT(?prevRep) as ?result)
    }
    ORDER BY ASC(?result)
"""
list(store.query(query))

[<QuerySolution result=<NamedNode value=http://example.org/picture_01>>,
 <QuerySolution result=<NamedNode value=http://example.org/picture_02>>]

In [11]:
# insert new representation
# set element representedBy it
# set representedBy link to active
# add "inactive" status to old representedBy

query = """
    PREFIX : <http://example.org/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
    
    INSERT {
        ?newRep a :Representation .

        << ?newRep a :Representation >>
            :representationType :picture ;

            :device "NikonD5600" ;

            :fileUrl "http://localhost:3000/uploads/pictures/picture_03.jpg" ;
            :pictureUrl "http://localhost:3000/uploads/pictures/picture_03.jpg" ;
            
            :createdBy "Luk Vermeulen" ;
            :creationDate ?currentDate ;
            
            :hasPreviousRepresentation << ?oldRep a :Representation >> .

        # representedBy link
        ?element :representedBy ?newRep .

        # new status information
        << ?element :representedBy ?newRep >>
            :date ?currentDate ;
            :hasStatus "active" .

        << << ?element :representedBy ?newRep >> :hasStatus "active" >>
            :date ?currentDate . 

        # add "inactive" to old representation
        << ?element :representedBy ?oldRep >>
            :hasStatus "inactive" .

        << << ?element :representedBy ?oldRep >> :hasStatus "inactive" >>
            :date ?currentDate .
    }
    WHERE {
        BIND(:picture_03 AS ?newRep) .
        BIND(:picture_02 AS ?oldRep) .
        BIND(:column_01 AS ?element) .
        BIND( xsd:dateTime(NOW()) AS ?currentDate ) .
    }
"""
store.update(query)

# show all active representations of element column_01
list(store.query(allActivePicRepsOfElement))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_03>>]

In [12]:
list(store.query(allPicsAndStatus))

[<QuerySolution rep=<NamedNode value=http://example.org/picture_02> status=<Literal value=inactive datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_03> status=<Literal value=active datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_a> status=<Literal value=inactive datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>,
 <QuerySolution rep=<NamedNode value=http://example.org/picture_b> status=<Literal value=active datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#string>>>]